# Aufgabe 2: Q-Learning Tic Tac Toe



Wir initialisieren Q(s,a) für alle möglichen Zustands- Aktions-Paare mit 0. Anfangs sind also alle möglichen Aktionen gleich wahrscheinlich.

Zur Optimierung der Laufzeit berechnen wir nicht in jedem Zustand neu alle möglichen Aktionen, sondern speichern für alle bereits getroffenen Zustände die dort möglichen Aktionen.

## Implementieren der Trainingsumgebung
Wir initialisieren ein Dictionary `Q_table`, in dem die bisher probierten Zustands- Aktions- Paare mit einer Wertung gespeichert werden.

Schlüssel sind dazu Tupel (Zustand, Aktion), wobei die Aktion eine Zahl von 0 bis 8 und Zustand ein Tupel mit 9 Elementen aus {0,1,2} sind.

Außerdem initialisieren wir `action_dict` um alle in einem Zustand (als Schlüssel gegeben) möglichen Aktionen zu speichern.

Die `eploration_rate` in der $n$-ten Episode ist $\varepsilon^n$.

In [7]:
def train_q_learning(learning_rate, discount_factor, base_exploration_rate, num_episodes=1e4, reward_dict={"win":1, "loss":-1, "draw":0, "move":-0.05}):
    """
    play Tic Tac Toe [num_episodes] times to learn using Q-Learning with the given learning rate and discount_factor.
    inputs:
        learning_rate - (float) in range [0,1] - alpha
        discount_factor - (float) in range [0,1] - gamma
        base_exploration_rate - (float) - the starting exploration rate
        num_episodes - (int) - number of episodes for training
        reward_dict - (dict) - a dictionary specifying the rewards for winning, losing and draw
            -> must have keys "win", "loss", "draw"
    returns:
        (dict) - the Q-table after training with the given parameters
    """
    Q_table = dict()    # assign values to every visited state-action pair
    N_table = dict()    # counting how often each state-action pair was visited
    action_dict = dict()    # save the possible actions for each state

    games = []
    exploration_rate = base_exploration_rate
    for n in range(num_episodes):
        # play episode
        state_hist = play_episode(Q_table, action_dict, exploration_rate, discount_factor, learning_rate, reward_dict, N_table)
        exploration_rate *= base_exploration_rate
        games.append(state_hist)

    print("final exploration rate:", exploration_rate)
    
    return Q_table, N_table, games
        

### Spielen einer Episode

Während dem Training spielt der Algorithmus gegen sich selbst und erforscht so gleichzeitig die Zustände, wenn er selbst anfängt und wenn der Gegner anfängt.

Durch festlegen des Zeichens des Startspielers ist in jedem Zustand eindeutig, welches Zeichen als nächstes gesetzt werden muss.

Zur späteren Aktualisierung der Q-Werte werden alle Zustände und gewählten Aktionen gespeichert (in den Listen `state_history` und `action_history`).

In [8]:
def play_episode(Q_table, action_dict, exploration_rate, discount_factor=0.95, learning_rate=0.1, reward_dict={"win":1, "loss":-1, "draw":0, "move":-0.05}, N_table=dict()):
    """
    self-play an entire episode
    returns:
        (list) - state history
        (list) - action history
    
    action_dict is changed in-place
    """ 
    field = [0 for _ in range(9)]
    sign = 1
    action_history = []
    state_history = []
    while True:
        state = tuple(field)
        state_history.append(state)
        # get possible actions
        try:
            actions = action_dict[state]
        except KeyError:
            actions = get_actions(field)
            action_dict[state] = actions

        if len(state_history) > 2: 
            # we know the state that resulted from the last action
            update_q_table(Q_table, state_history, action_history, actions, discount_factor, learning_rate, reward_dict, N_table)

        if len(actions) == 0:
            break # game has ended

        action = choose_action(state, actions, Q_table, exploration_rate=exploration_rate)
        action_history.append(action)
        field[action] = sign
        sign = sign%2 + 1 # toggle sign between 1 and 2

    last_state = state_history[-2]
    last_action = action_history[-1]
    if not (last_state, last_action) in Q_table.keys():
        Q_table[(last_state, last_action)] = 0
        N_table[(last_state, last_action)] = 0
    # print("before", Q_table[(last_state, last_action)])
    Q_table[(last_state, last_action)] += learning_rate*(reward_dict["win"] - Q_table[(last_state, last_action)])
    N_table[(last_state, last_action)] += 1
    
    return state_history


def update_q_table(Q_table, state_history, action_history, actions, discount_factor, learning_rate, reward_dict, N_table):
    """
    update the second to last state in the Q-table
    returns:
        None
    """
    prev_state = state_history[-3] # S = state
    prev_action = action_history[-2] # A = action
    state = state_history[-1] # S' = next state after action A

    reward = get_reward(list(state), actions, reward_dict) # R = Reward
    next_rewards = [] # Q(S', a') for all actions a'
    for action in actions:
        try:
            next_rewards.append(Q_table[(state, action)])
        except KeyError:
            Q_table[(state, action)] = 0
            N_table[(state, action)] = 0
            next_rewards.append(0)

    if not (prev_state, prev_action) in Q_table.keys():
        Q_table[(prev_state, prev_action)] = 0
        N_table[(prev_state, prev_action)] = 0
    if ((1,2,0,0,1,2,0,0,0),8) in Q_table.keys():
        test_value = str(Q_table[((1,2,0,0,1,2,0,0,0),8)])
    # Q(S,A) += alpha*(R + gamma * max(S', a') - Q(S,A))
    Q_table[(prev_state, prev_action)] += learning_rate*(reward + discount_factor * max(next_rewards, default=0) - Q_table[(prev_state, prev_action)])
    N_table[(prev_state, prev_action)] += 1
    if ((1,2,0,0,1,2,0,0,0),8) in Q_table.keys():
        if test_value != str(Q_table[((1,2,0,0,1,2,0,0,0),8)]):
            print("old value:", test_value)
            print("reward was", reward)
            print("new value:", Q_table[((1,2,0,0,1,2,0,0,0),8)])


def get_reward(field, actions, reward_dict):
    """
    return the reward for the given field and possible actions
    """
    if len(actions) > 0:
        reward = reward_dict["move"]
    else:
        winner = game_ended(field, get_winner=True)
        if winner == 0: #draw
            reward = reward_dict["draw"]
        else:
            reward = reward_dict["loss"]
    return reward

### Wählen einer Aktion
Zum Wählen der Aktion wird eine $\varepsilon$-greedy Strategie genutzt.

Wird damit ausgewählt bekanntes Wissen zu nutzen (Exploitation), so wird, falls es mehrere Aktionen mit maximalem Wert gibt, eine zufällige davon ausgewählt.

In [9]:
import random
def choose_action(state, actions, Q_table, exploration_rate=0):
    """
    choose an action based on the possible actions, the current Q-table and the current exploration rate
    """
    r = random.random()
    if r > exploration_rate:
        # print("exploit", r, exploration_rate)
        # exploit knowledge
        action_values = []
        for action in actions:
            try:
                action_values.append(Q_table[(state,action)])
            except KeyError:
                action_values.append(0)
        max_value = max(action_values)
        best_actions = []
        for action, value in zip(actions, action_values):
            if value == max_value:
                best_actions.append(action)
        # return random action with maximum expected reward
        return random.choice(best_actions)
    # explore environment through random move
    return random.choice(actions)

## Anwenden des Algorithmus

Mit all diesen Funktionen können wir nun endlich den Algorithmus testen. Es epfiehlt sich den Agent mindestens über 10.000 Spiele zu trainieren, dies sollte nur wenige Sekunden dauern. Die Trainingszeit steigt wie zu erwarten ziemlich linear mit der Anzahl an Trainingsspielen.

Im Folgenden wird die exploration_rate mit `1-(5/num_episodes)` so gewählt, dass sie am Ende des Trainings nahe 0 kommt, aber nicht zu lange so klein ist.

Jeder nicht-terminal Zug wird mit einem kleinen Wert bestraft um schnelles gewinnen zu fördern.

Es ist schwierig eine optimale Lernrate $\alpha$ oder einen besonders guten discount_factor $\gamma$ zu finden, da das Evaluieren, wie gut die KI nach dem Training noch nicht automatisiert ist.

Mit mehr Zeit wäre es denkbar genetische Algorithmen für diese Werte anzuwenden und die trainierten Agenten gegeneinander spielen zu lassen. So könnte man bessere Startwerte finen.
Dabei würden relativ kurze Trainingszeiten verwendet werden um dann mit den gefundenen, optimierten Startwerten hoffentlich einen besseren Agenten trainieren zu können.

In [13]:
learning_rate = 0.01
discount_factor = 0.95
num_episodes = int(1e5)
exploration_rate = 1-(5/num_episodes)
reward_dict = {"win":2,      # reward for win
               "loss":-1,    # reward for loss
               "draw":0,     # reward for draw
               "move":-0.05} # reward per non-terminal move

%time Q_table, N_table, games = train_q_learning(learning_rate, discount_factor, exploration_rate, num_episodes=num_episodes, reward_dict=reward_dict)
print(len(Q_table))
print(len(N_table))

final exploration rate: 0.00673676792504109
Wall time: 15 s
16164
16164


In [15]:
def analyse_N_table(N_table):
    n_games = len(N_table)
    max_n = (0,float("-inf"))
    min_n = (0,float("inf"))

    n_sum = 0
    for key,value in N_table.items():
        if value < min_n[1]:
            min_n = (key,value)
        if value > max_n[1]:
            max_n = (key,value)
        n_sum += value
    avg_n = n_sum/n_games
    
    print(f"average vists per state: {avg_n}")
    print(f"max visits: {max_n[1]} for state {max_n[0]}")
    print(f"min visits: {min_n[1]} for state {min_n[0]}")

analyse_N_table(N_table)

average vists per state: 50.27140559267508
max visits: 81462 for state ((0, 0, 0, 0, 0, 0, 0, 0, 0), 2)
min visits: 0 for state ((0, 1, 2, 1, 0, 0, 2, 1, 2), 5)


In [65]:
def export_Q_table(Q_table, filename="Q_table.txt"):
    """
    write the given Q-table into a file
    """
    with open(filename, "w") as file:
        file.write("Q_table = {\n")
        for key, value in Q_table.items():
            file.write(str(key) + ":" + str(value) + ",\n")
        file.write("}")

In [86]:
def import_Q_table(filename="Q_table.txt"):
    """
    import Q_table as dictionary:
    keys are state-action pairs as a tuple of a tuple (9 integers: 0/1/2) and an integer (0-8)
    values are the corresponding Q-values

    Example:
        Q_table[((0,0,0,0,1,0,0,0,0),2)] -> 0.3
    """
    Q_table = dict()
    with open(filename, "r") as file:
        for line in file.readlines():
            if line == "Q_table = {\n" or line == "}":
                continue
            state_action, value = line[:-2].split(":")
            state = tuple([int(x.strip(" ")) for x in state_action[2:-5].split(",")])
            action = int(state_action[-2])
            Q_table[(state, action)] = float(value)
    return Q_table

test = import_Q_table()
export_Q_table(test, filename="Q_table2.txt")

In [66]:
export_Q_table(Q_table)

Wir können uns einzelne Spiele aus dem Trainingsprozess ansehen:

In [8]:
def show_game(games, i):
    for state in games[i]:
        print_field(state)
        # print(game_ended(list(state)))
    print("#"*25)
# show_game(games, 0)
show_game(games, num_episodes-50)

-------------
|   |   |   |
-------------
|   |   |   |
-------------
|   |   |   |
-------------
-------------
|   |   |   |
-------------
|   | o |   |
-------------
|   |   |   |
-------------
-------------
|   |   |   |
-------------
|   | o |   |
-------------
| x |   |   |
-------------
-------------
|   | o |   |
-------------
|   | o |   |
-------------
| x |   |   |
-------------
-------------
|   | o |   |
-------------
|   | o |   |
-------------
| x | x |   |
-------------
-------------
|   | o |   |
-------------
|   | o |   |
-------------
| x | x | o |
-------------
-------------
| x | o |   |
-------------
|   | o |   |
-------------
| x | x | o |
-------------
-------------
| x | o |   |
-------------
| o | o |   |
-------------
| x | x | o |
-------------
-------------
| x | o | x |
-------------
| o | o |   |
-------------
| x | x | o |
-------------
-------------
| x | o | x |
-------------
| o | o | o |
-------------
| x | x | o |
-------------
####################

## Testen durch Menschen

Mit der folgenden Funktion können wir auch die KI testen indem wir selbst gegen sie spielen. Dabei werden die Züge ausschließlich nach exploitation gewählt, also immer ein optimaler Zug entsprechend den Q-Werten aus der tabelle `Q_table`.

In [52]:
def play_AI(Q_table):
    start_player = ""
    while not start_player in ["me", "ai"]:
        start_player = input("Who starts? (me/ ai)\n")
    
    field = [0 for _ in range(9)]
    sign = 1
    print_field(field)

    playing = True
    while playing:
        actions = get_actions(field)
        if start_player == "ai":
            action = choose_action(tuple(field), actions, Q_table, exploration_rate=0)
            # print("action had value:", Q_table[(tuple(field), action)])
        else:
            action = -5
            while not action in actions:
                action = input(f"Choose your action ({str(actions)[1:-1]})\n")
                if action == "end":
                    playing = False
                    break
                try:
                    action = int(action)
                except:
                    pass
        if not playing:
            print("game ended")
            break
        field[action] = sign
        sign = sign%2 + 1
        print_field(field)
        start_player = "ai" if start_player == "me" else "me"

        winner = game_ended(field,get_winner=True)
        if winner != None:
            playing = False
            if winner == 0:
                print("draw!")
            elif winner == 1:
                print("'o' won!")
            else:
                print("'x' won!")

play_AI(Q_table_1)

-------------
|   |   |   |
-------------
|   |   |   |
-------------
|   |   |   |
-------------
-------------
| o |   |   |
-------------
|   |   |   |
-------------
|   |   |   |
-------------
-------------
| o |   |   |
-------------
|   |   | x |
-------------
|   |   |   |
-------------
-------------
| o |   |   |
-------------
|   | o | x |
-------------
|   |   |   |
-------------
-------------
| o |   | x |
-------------
|   | o | x |
-------------
|   |   |   |
-------------
-------------
| o |   | x |
-------------
|   | o | x |
-------------
|   |   | o |
-------------
'o' won!


Selbst nach 1.000.000 Trainingsspielen kann man, mit etwas Glück, noch gegen den Algorithmus gewinnen.

Die wenig (<=100 Spiele) trainierte KI ist sehr einfach zu schlagen. Dennoch scheint diese gegen die mehr trainierte KI zu gewinnen.

In [23]:
field = (1,2,0,1,1,0,2,1,2)
print_field(field)
for action in get_actions(list(field)):
    print(f"action {action} has value {Q_table[(field,action)]}")

-------------
| o | x |   |
-------------
| o | o |   |
-------------
| x | o | x |
-------------
action 2 has value 0.0
action 5 has value 0.0


## Teste zwei unterschiedliche KIs



In [19]:
def test_ais(Q_table_1, Q_table_2, n_games=100, exploration_rate=0.1):
    results = [0,0,0] # AI-1 victories, AI-2 victories, draws
    for n in range(n_games):
        field = [0]*9
        active_index = n%2
        Q_tables = (Q_table_1, Q_table_2)
        sign = 1
        winner = None
        while winner==None:
            actions = get_actions(field)
            if actions == []:
                print_field(field)
            action = choose_action(tuple(field), actions, Q_tables[active_index], exploration_rate=exploration_rate)
            field[action] = sign
            sign = sign%2 + 1
            winner = game_ended(field, get_winner=True)
        if winner == 0:
            results[2] += 1
        elif n%2 == 0:
            if winner == 1:
                results[0] += 1
            else:
                results[1] += 1
        elif n%2 == 1:
            if winner == 1:
                results[1] += 1
            else:
                results[0] += 1

    print("KI 1 hat {} Mal gewonnen.\nKI 2 hat {} Mal gewonnen.\nEs gab {} Unentschieden.".format(*results))

### train AI 1

In [26]:
# KI 1
learning_rate = 0.05
discount_factor = 0.80
num_episodes = int(1e0)
exploration_rate = 1-(5/num_episodes)
reward_dict = {"win":1,      # reward for win
               "loss":-2,    # reward for loss
               "draw":0.5,     # reward for draw
               "move":0} # reward per non-terminal move

%time Q_table_1, N_table_1, _ = train_q_learning(learning_rate, discount_factor, exploration_rate, num_episodes=num_episodes, reward_dict=reward_dict)
analyse_N_table(N_table_1)

final exploration rate: 16.0
Wall time: 23.5 ms
average vists per state: 0.27586206896551724
max visits: 1 for state ((0, 1, 0, 2, 0, 0, 0, 0, 0), 2)
min visits: 0 for state ((0, 1, 0, 2, 0, 0, 0, 0, 0), 0)


### train AI 2

In [43]:
# KI 2

learning_rate = 0.05
discount_factor = 0.75
num_episodes = int(1e3)
exploration_rate = 1-(3/num_episodes)
reward_dict = {"win":1,      # reward for win
               "loss":-1,    # reward for loss
               "draw":0,     # reward for draw
               "move":-0.05} # reward per non-terminal move

%time Q_table_2, N_table_2, _ = train_q_learning(learning_rate, discount_factor, exploration_rate, num_episodes=num_episodes, reward_dict=reward_dict)
analyse_N_table(N_table_2)

final exploration rate: 0.049414393574685855
Wall time: 299 ms
average vists per state: 0.6824988516306845
max visits: 115 for state ((0, 0, 0, 0, 0, 0, 0, 0, 0), 6)
min visits: 0 for state ((1, 0, 0, 0, 0, 0, 2, 1, 0), 2)


### Play AI 1 vs. AI 2

In [48]:
test_ais(Q_table_1, Q_table_2, n_games=10_000, exploration_rate=0)

KI 1 hat 4550 Mal gewonnen.
KI 2 hat 4192 Mal gewonnen.
Es gab 1258 Unentschieden.


Auch nach Ausprobieren zahlreicher Werte für $\alpha$, $\gamma$, $\varepsilon$ und für verschiedene Rewards konnte ich keine Kombination finden, die die hier zweite KI konstant schlägt.

Interessanterweise führt aber eine geringere Anzahl an Trainingsspielen zu mehr Siegen. Das zeigt deutlich, dass die KIs auch nach längerem Selbsttraining nicht perfekt sind. Eine genaue Erklärung für dieses Verhalten habe ich aber nicht gefunden.

Die `exploration_rate` in den Testspielen hat kaum Einfluss auf das Sieg-Verhältnis, nur auf die gesamte Anzahl an Unentschieden.